In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from models.option_pricing_models import BlackScholesModel, BinomialTreeModel, SABRModel


In [ ]:

ticker = "AAPL"
expiration = "2024-12-20"
opt = yf.Ticker(ticker)
calls = opt.option_chain(expiration).calls
spot_price = opt.history(period="1d")['Close'].iloc[-1]


In [ ]:

calls = calls[['strike', 'impliedVolatility', 'bid', 'ask']].dropna()
calls = calls[calls['impliedVolatility'] > 0]
calls = calls[calls['bid'] > 0]
calls['MidPrice'] = (calls['bid'] + calls['ask']) / 2
T = (pd.to_datetime(expiration) - pd.Timestamp.today()).days / 365
r = 0.04
sigma_col = 'impliedVolatility'


In [ ]:

bs_model = BlackScholesModel(r=r)
bt_model = BinomialTreeModel(r=r, steps=100)
sabr_model = SABRModel(alpha=0.3, beta=0.5, rho=-0.3, nu=0.5, r=r)


In [ ]:

def compute_errors(df, spot, T, r):
    bs_prices, bt_prices, sabr_prices = [], [], []
    for _, row in df.iterrows():
        K = row['strike']
        sigma = row[sigma_col]
        market_price = row['MidPrice']

        bs_price = bs_model.price(spot, K, T, sigma, option_type='call')
        bt_price = bt_model.price(spot, K, T, sigma, option_type='call')
        sabr_price = sabr_model.price(spot, K, T, option_type='call')

        bs_prices.append(bs_price)
        bt_prices.append(bt_price)
        sabr_prices.append(sabr_price)

    df['BSPrice'] = bs_prices
    df['BTPrice'] = bt_prices
    df['SABRPrice'] = sabr_prices

    for col in ['BSPrice', 'BTPrice', 'SABRPrice']:
        df[f'{col}_RelError'] = np.abs(df[col] - df['MidPrice']) / df['MidPrice']

    return df

calls = compute_errors(calls, spot_price, T, r)
calls[['strike', 'MidPrice', 'BSPrice_RelError', 'BTPrice_RelError', 'SABRPrice_RelError']].head()


In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(calls['strike'], calls['BSPrice_RelError'], label='Black-Scholes', marker='o')
plt.plot(calls['strike'], calls['BTPrice_RelError'], label='Binomial Tree', marker='x')
plt.plot(calls['strike'], calls['SABRPrice_RelError'], label='SABR', marker='s')
plt.xlabel("Strike Price")
plt.ylabel("Relative Error")
plt.title("Relative Pricing Error by Model")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
